In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-08 17:01:38--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.52MB/s    in 0.2s    

2022-01-08 17:01:39 (4.52 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
import pandas as pd
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Automotive_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df=spark.read.csv(SparkFiles.get("amazon_reviews_us_Automotive_v1_00.tsv.gz"),sep="\t",header=True)


In [ ]:
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36075342| RAB23OVFNCXZQ|B00LPRXQ4Y|     339193102|17" 2003-2006 For...|      Automotive|          1|            0|          0|   N|                Y|     As it was used,|As it was used, t...| 2015-08-31|
|         US|   42462164|R3NORADVJO6IE6|B000C7S0TO|     907684644|Spectra Premium C...|      Automotive|          5|    

In [ ]:
print(f'The row count is {df.count()}')

The row count is 3514942


In [ ]:
df_duplicates_cleaned=df.drop_duplicates()
df_drop_na=df_duplicates_cleaned.dropna()


In [ ]:
df_sql_column_auto=df_drop_na.select(['review_id','customer_id','product_id','product_parent','review_date'])

df_sql_column_auto.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R1009QQKORHRM|   32368402|B009XOZL9O|     995466732| 2014-06-25|
|R100S2N5S5N7W8|   40930386|B001F73234|     367753376| 2009-04-19|
|R100UUUPVZTPN1|    2068235|B000WJIMPY|     237873710| 2015-02-06|
|R101FR7DLA44VH|    4576541|B009DP32CG|     466224782| 2013-05-23|
|R101PDT966HA58|   19635680|B004L04TD4|     727205780| 2015-04-30|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
mode="append"
jdbc_url=""
config={"user":'',
        "password":'',
        "driver":'org.postgresql.Driver'}

In [ ]:
df_sql_column_auto.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
from pyspark.sql.functions import desc

customers_df_auto = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df_auto = customers_df_auto.orderBy(desc("count(customer_id)"))
customers_df_auto = customers_df_auto.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df_auto.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   18307436|           274|
|   27969465|           258|
|   34741901|           224|
|   44332703|           207|
|   31142840|           183|
|   46796594|           182|
|   12626911|           168|
|   30923288|           165|
|   52263513|           157|
|    2981713|           155|
|   43953318|           151|
|   14891799|           144|
|   35292542|           141|
|   36179085|           140|
|   44586096|           137|
|   18411963|           134|
|    1077707|           133|
|   15171348|           130|
|   21710957|           129|
|   17233732|           127|
+-----------+--------------+
only showing top 20 rows



In [ ]:
products_df_auto = df.select(["product_id","product_title"])
products_df_auto = products_df_auto.dropDuplicates()
products_df_auto.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B002OKKCO8|06-09 KAWASAKI ZX...|
|B00OZXP3BY|Oem Factory Stock...|
|B00HA26HZS|Suzuki Hayabusa G...|
|B00BITHKD2|PT Auto Warehouse...|
|B000688VNE|Hi-Lift Jack Hi-L...|
|B004GCP4GS|OE Replacement Au...|
|B0006HPA02|MSD 32829 8.5mm S...|
|B00EK6NS22|uxcell® Car Bike ...|
|B00WMR1I64|MICTUNING mt00880...|
|B009S31VVW|Dorman 81702 Ford...|
|B00QJ8FA70|Parts Player New ...|
|B00HZOCCBY|MAXFLOORMAT Floor...|
|B00DZPC5NM|Dodge Grand Carav...|
|B001VKT9IW|HushMat 10301 Ult...|
|B009SEJI66|2011-2013 Toyota ...|
|B007QBKODO|ACDelco 18A2729 P...|
|B007LIBJPO|Custom Accessorie...|
|B009YKQR2C|Zone Tech 4pcs 20...|
|B00062YAZ8|Edelbrock 4201 El...|
|B008IAHBWY|Jeep Wrangler Tj ...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
df_sql_column_auto.write.jdbc(url=jdbc_url, table='df_sql_column_auto', mode=mode, properties=config)

In [ ]:
products_df_auto.write.jdbc(url=jdbc_url, table='products_auto', mode=mode, properties=config)

In [ ]:
customers_df_auto.write.jdbc(url=jdbc_url, table='customers_auto', mode=mode, properties=config)